In [33]:
import requests
import pandas as pd

session_key = 9197
BASE_URL = "https://api.openf1.org/v1"
start_time = "2023-11-26T13:04:55.111000+00:00"
end_time = "2023-11-26T13:09:55.111000+00:00"
PARAMS = f"date>={start_time}&date<={end_time}"




In [34]:
def fetch_all_data():
    print("Fetching drivers...")
    try:
        get_drivers_url = f"{BASE_URL}/drivers?session_key={session_key}"
        drivers = requests.get(get_drivers_url).json()
        print(f"Fetched {len(drivers)} drivers.")
    except Exception as e:
        print(f"Error fetching drivers: {e}")
        return [], [], [], [], []  # Return empty lists if drivers fail to fetch
    
    car_data_all = []
    location_data_all = []
    interval_data_all = []
    position_data_all = []

    for driver in drivers:
        driver_number = driver.get('driver_number', None)
        driver_name = driver.get('broadcast_name', "Unknown")
        
        if not driver_number:
            print(f"Skipping driver with missing driver_number: {driver}")
            continue
        
        print(f"Fetching data for driver {driver_name} (driver_number={driver_number})...")
        
        # Fetch car data
        try:
            get_car_data_url = f"{BASE_URL}/car_data?session_key={session_key}&speed>=0&driver_number={driver_number}&{PARAMS}"
            car_data = requests.get(get_car_data_url).json()
            car_data_all.extend(car_data)
            print(f"Fetched {len(car_data)} car data entries for driver {driver_number}.")
        except Exception as e:
            print(f"Error fetching car data for driver {driver_number}: {e}")
        
        # Fetch location data
        try:
            get_location_data_url = f"{BASE_URL}/location?session_key={session_key}&driver_number={driver_number}&{PARAMS}"
            location_data = requests.get(get_location_data_url).json()
            location_data_all.extend(location_data)
            print(f"Fetched {len(location_data)} location data entries for driver {driver_number}.")
        except Exception as e:
            print(f"Error fetching location data for driver {driver_number}: {e}")
        
        # Fetch interval data
        try:
            get_interval_data_url = f"{BASE_URL}/intervals?session_key={session_key}&driver_number={driver_number}&{PARAMS}"
            interval_data = requests.get(get_interval_data_url).json()
            interval_data_all.extend(interval_data)
            print(f"Fetched {len(interval_data)} interval data entries for driver {driver_number}.")
        except Exception as e:
            print(f"Error fetching interval data for driver {driver_number}: {e}")
        
        # Fetch position data
        try:
            get_position_data_url = f"{BASE_URL}/position?session_key={session_key}&driver_number={driver_number}&{PARAMS}"
            position_data = requests.get(get_position_data_url).json()
            position_data_all.extend(position_data)
            print(f"Fetched {len(position_data)} position data entries for driver {driver_number}.")
        except Exception as e:
            print(f"Error fetching position data for driver {driver_number}: {e}")
    
    return drivers, car_data_all, location_data_all, interval_data_all, position_data_all


# Function to construct the leaderboard



In [41]:
def construct_leaderboard(drivers, car_data_all, location_data_all, interval_data_all, position_data_all):
    print("Constructing leaderboard...")

    # Map drivers by driver_number for quick access
    driver_map = {driver['driver_number']: driver for driver in drivers}

    # Group data by driver_number for efficient timestamp comparison
    car_data_by_driver = {driver['driver_number']: sorted([entry for entry in car_data_all if entry['driver_number'] == driver['driver_number']], key=lambda x: x['date']) for driver in drivers}
    location_data_by_driver = {driver['driver_number']: sorted([entry for entry in location_data_all if entry['driver_number'] == driver['driver_number']], key=lambda x: x['date']) for driver in drivers}
    interval_data_by_driver = {driver['driver_number']: sorted([entry for entry in interval_data_all if entry['driver_number'] == driver['driver_number']], key=lambda x: x['date']) for driver in drivers}
    position_data_by_driver = {driver['driver_number']: sorted([entry for entry in position_data_all if entry['driver_number'] == driver['driver_number']], key=lambda x: x['date']) for driver in drivers}

    # Find common timestamps
    all_timestamps = set(entry['date'] for entry in location_data_all)
    for driver_number in driver_map.keys():
        driver_timestamps = set(entry['date'] for entry in location_data_by_driver[driver_number])
        all_timestamps &= driver_timestamps  # Keep only timestamps present for all drivers

    all_timestamps = sorted(all_timestamps)  # Sort timestamps

    # Helper function to find the closest entry
    def find_closest(data_list, timestamp):
        closest = None
        for entry in data_list:
            if entry['date'] <= timestamp:
                closest = entry
            else:
                break
        return closest

    leaderboard = []

    # Construct leaderboard for each common timestamp
    for timestamp in all_timestamps:
        timestamp_entry = {"timestamp": timestamp, "drivers": []}

        for driver_number, driver in driver_map.items():
            # Get data for the current timestamp or closest match
            car_data = find_closest(car_data_by_driver[driver_number], timestamp)
            location_data = next((entry for entry in location_data_by_driver[driver_number] if entry['date'] == timestamp), None)
            position_data = find_closest(position_data_by_driver[driver_number], timestamp)
            interval_data = find_closest(interval_data_by_driver[driver_number], timestamp)

            # Add driver data to the timestamp entry
            timestamp_entry["drivers"].append({
                "driver": driver,
                "car_data": car_data,
                "location_data": location_data,
                "position_data": position_data,
                "interval_data": interval_data,
            })

        leaderboard.append(timestamp_entry)

    print("Leaderboard construction complete.")
    return leaderboard


In [36]:
# Main execution in the Jupyter Notebook
print("Starting data fetching process...")
drivers, car_data_all, location_data_all, interval_data_all, position_data_all = fetch_all_data()
print("Data fetching complete.")



Starting data fetching process...
Fetching drivers...
Fetched 20 drivers.
Fetching data for driver M VERSTAPPEN (driver_number=1)...
Fetched 1138 car data entries for driver 1.
Fetched 1167 location data entries for driver 1.
Fetched 4 interval data entries for driver 1.
Fetched 0 position data entries for driver 1.
Fetching data for driver L SARGEANT (driver_number=2)...
Fetched 1140 car data entries for driver 2.
Fetched 1168 location data entries for driver 2.
Fetched 80 interval data entries for driver 2.
Fetched 3 position data entries for driver 2.
Fetching data for driver D RICCIARDO (driver_number=3)...
Fetched 1139 car data entries for driver 3.
Fetched 1168 location data entries for driver 3.
Fetched 80 interval data entries for driver 3.
Fetched 2 position data entries for driver 3.
Fetching data for driver L NORRIS (driver_number=4)...
Fetched 1140 car data entries for driver 4.
Fetched 1167 location data entries for driver 4.
Fetched 81 interval data entries for driver 4.


In [43]:
import json

print("Starting leaderboard construction...")
print(interval_data_all)
# leaderboard = construct_leaderboard(drivers, car_data_all, location_data_all, interval_data_all, position_data_all)

# # Display the leaderboard as JSON in the notebook
# print("Leaderboard:")
# print(json.dumps(leaderboard, indent=4))

# # Optionally save the leaderboard to a file
# with open("leaderboard.json", "w") as f:
#     json.dump(leaderboard, f, indent=4)
# print("Leaderboard saved to leaderboard.json.")

Starting leaderboard construction...
[{'gap_to_leader': None, 'interval': None, 'driver_number': 1, 'date': '2023-11-26T13:04:58.064000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': None, 'interval': None, 'driver_number': 1, 'date': '2023-11-26T13:06:28.813000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': None, 'interval': None, 'driver_number': 1, 'date': '2023-11-26T13:07:59.218000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': None, 'interval': None, 'driver_number': 1, 'date': '2023-11-26T13:09:29.936000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': 8.871, 'interval': 1.012, 'driver_number': 2, 'date': '2023-11-26T13:04:56.346000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': 9.204, 'interval': 1.245, 'driver_number': 2, 'date': '2023-11-26T13:05:00.361000+00:00', 'session_key': 9197, 'meeting_key': 1226}, {'gap_to_leader': 9.788, 'interval': 1.627, 'driver_number': 2, 

In [ ]:
[
    {
        "2023-11-26T12:01:02.527000+00:00": {
            "driver": "<driver_info",
            "date" : "<date",
            "location": "<location_data>",
            "position": "<position_data>",
            "interval": "<interval>"

        }
    }
]

In [49]:
def fetch_and_construct_leaderboard(BASE_URL, session_key):
    print("Fetching drivers...")
    try:
        get_drivers_url = f"{BASE_URL}/drivers?session_key={session_key}"
        drivers = requests.get(get_drivers_url).json()
        print(f"Fetched {len(drivers)} drivers.")
    except Exception as e:
        print(f"Error fetching drivers: {e}")
        return []

    leaderboard = []
    all_timestamps = None  # Will hold common timestamps across all drivers
    driver_data_map = {}  # Store all data per driver for quick access

    for driver in drivers:
        driver_number = driver.get("driver_number")
        driver_name = driver.get("broadcast_name", "Unknown")
        print(f"Fetching data for driver {driver_name} (driver_number={driver_number})...")
        
        try:
            # Fetch car data
            get_car_data_url = f"{BASE_URL}/car_data?session_key={session_key}&speed>=0&driver_number={driver_number}&{PARAMS}"
            car_data = requests.get(get_car_data_url).json()
            print(f"Fetched {len(car_data)} car data entries for driver {driver_number}.")

            # Fetch location data
            get_location_data_url = f"{BASE_URL}/location?session_key={session_key}&driver_number={driver_number}&{PARAMS}"
            location_data = requests.get(get_location_data_url).json()
            print(f"Fetched {len(location_data)} location data entries for driver {driver_number}.")

            # Fetch interval data
            get_interval_data_url = f"{BASE_URL}/intervals?session_key={session_key}&driver_number={driver_number}"
            interval_data = requests.get(get_interval_data_url).json()
            print(f"Fetched {len(interval_data)} interval data entries for driver {driver_number}.")

            # Fetch position data
            get_position_data_url = f"{BASE_URL}/position?session_key={session_key}&driver_number={driver_number}"
            position_data = requests.get(get_position_data_url).json()
            print(f"Fetched {len(position_data)} position data entries for driver {driver_number}.")

            # Store data
            driver_data_map[driver_number] = {
                "driver": driver,
                "car_data": sorted(car_data, key=lambda x: x['date']),
                "location_data": sorted(location_data, key=lambda x: x['date']),
                "interval_data": sorted(interval_data, key=lambda x: x['date']),
                "position_data": sorted(position_data, key=lambda x: x['date']),
            }

            # Update common timestamps
            driver_timestamps = set(entry["date"] for entry in location_data)
            if all_timestamps is None:
                all_timestamps = driver_timestamps
            else:
                all_timestamps &= driver_timestamps  # Intersection of timestamps
        except Exception as e:
            print(f"Error fetching data for driver {driver_number}: {e}")

    if not all_timestamps:
        print("No common timestamps found.")
        return []

    all_timestamps = sorted(all_timestamps)  # Sort common timestamps

    # Helper function to find the closest data
    def find_closest(data_list, timestamp):
        closest = None
        for entry in data_list:
            if entry["date"] <= timestamp:
                closest = entry
            else:
                break
        return closest

    # Helper function to find the last known position or interval
    def find_last_known(data_list, timestamp):
        last_known = None
        for entry in data_list:
            if entry["date"] <= timestamp:
                last_known = entry
            else:
                break
        return last_known

    print("Constructing leaderboard...")
    for timestamp in all_timestamps:
        timestamp_entry = {"timestamp": timestamp, "drivers": []}

        for driver_number, data in driver_data_map.items():
            car_data = find_closest(data["car_data"], timestamp)
            location_data = next((entry for entry in data["location_data"] if entry["date"] == timestamp), None)
            position_data = find_last_known(data["position_data"], timestamp)
            interval_data = find_last_known(data["interval_data"], timestamp)

            timestamp_entry["drivers"].append({
                "driver": data["driver"],
                "car_data": car_data,
                "location_data": location_data,
                "position_data": position_data,
                "interval_data": interval_data,
            })

        leaderboard.append(timestamp_entry)

    print("Leaderboard construction complete.")
    return leaderboard


# Example Usage

leaderboard = fetch_and_construct_leaderboard(BASE_URL, session_key)

# Display the leaderboard or save it as needed
import json
print(json.dumps(leaderboard, indent=4))

# Optionally save to a file
with open("leaderboard.json", "w") as f:
    json.dump(leaderboard, f, indent=4)
print("Leaderboard saved to leaderboard.json.")


Fetching drivers...
Fetched 20 drivers.
Fetching data for driver M VERSTAPPEN (driver_number=1)...
Fetched 1138 car data entries for driver 1.
Fetched 1167 location data entries for driver 1.
Fetched 192 interval data entries for driver 1.
Fetched 11 position data entries for driver 1.
Fetching data for driver L SARGEANT (driver_number=2)...
Fetched 1140 car data entries for driver 2.
Fetched 1168 location data entries for driver 2.
Fetched 1400 interval data entries for driver 2.
Fetched 31 position data entries for driver 2.
Fetching data for driver D RICCIARDO (driver_number=3)...
Fetched 1139 car data entries for driver 3.
Fetched 1168 location data entries for driver 3.
Fetched 1391 interval data entries for driver 3.
Fetched 34 position data entries for driver 3.
Fetching data for driver L NORRIS (driver_number=4)...
Fetched 1140 car data entries for driver 4.
Fetched 1167 location data entries for driver 4.
Fetched 1383 interval data entries for driver 4.
Fetched 28 position dat